# 在 Bedrock AgentCore Runtime 中部署 MCP 服务器

本研讨会演示如何使用 Amazon Bedrock AgentCore Runtime 部署和使用模型上下文协议（MCP）服务器，为 AI 代理实现可扩展和安全的自定义工具部署。

## 概述

在本实验中，您将：
- 创建具有网络搜索功能的自定义 MCP 服务器
- 使用 Amazon Cognito 设置身份验证
- 将 MCP 服务器部署到 Bedrock AgentCore Runtime
- 使用 Strands Agents 测试已部署的服务器
- 了解清理程序

## 先决条件

开始本实验之前，请确保您具备：
- 已配置 AWS 凭证（IAM 角色或环境变量）
- 已安装所需的 Python 包
- 对 MCP 和 Bedrock AgentCore 概念有基本了解

如果您没有在假设 IAM 角色的环境中运行，请将您的 AWS 凭证设置为环境变量：

In [1]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

为 MCP 服务器开发、Strands Agents 和 Bedrock AgentCore SDK 安装所需的包：

In [2]:
#%pip install -q ddgs mcp strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit 

## 什么是用于 MCP 的 Bedrock AgentCore Runtime？

Amazon Bedrock AgentCore Runtime 允许您将模型上下文协议（MCP）服务器部署为托管的可扩展服务。主要优势包括：

- **可扩展性**：根据需求自动扩展
- **安全性**：内置身份验证和授权
- **托管基础设施**：无需管理服务器或容器
- **集成**：与 Bedrock 服务无缝集成

MCP 服务器提供 AI 代理可以用来扩展其功能的工具和资源，例如网络搜索、数据库访问或自定义业务逻辑。

### 创建自定义 MCP 服务器

让我们创建一个使用 DuckDuckGo 提供网络搜索功能的简单 MCP 服务器。该服务器将部署到 AgentCore Runtime 以供可扩展使用。

In [3]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# Define a websearch tool
@mcp.tool()
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> list:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Writing mcp_server.py


In [4]:
%%writefile requirements.txt
ddgs
mcp
bedrock-agentcore

Writing requirements.txt


## MCP 服务器的本地测试

在部署到 AgentCore Runtime 之前，先在本地测试 MCP 服务器。

### 步骤 1：启动 MCP 服务器

在终端中运行 MCP 服务器：

```bash
uv pip install -r requirements.txt
uv run mcp_server.py
```
or
```bash
pip install -r requirements.txt
python mcp_server.py
```

### 步骤 2：使用 Strands Agent 测试

执行以下代码来测试集成：

In [5]:
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# Connect to the web search MCP server
print("\nConnecting to MCP Server...")
mcp_url = f"http://localhost:8000/mcp"
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url))

with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"Available MCP tools: {[tool.tool_name for tool in mcp_tools]}")

    # Create agent with custom MCP tools
    agent = Agent(
        model="us.amazon.nova-pro-v1:0",  # Optional: Specify the model ID
        system_prompt="You are a helpful assistant that provides concise responses.",
        tools=mcp_tools,
    )

    response = agent("介绍 Bedrock AgentCore 是甚么")
    print(response) 


Connecting to MCP Server...
Available MCP tools: ['websearch']
<thinking> To provide an introduction to Bedrock AgentCore, I need to search for relevant information about it. I will use the websearch tool to find details about Bedrock AgentCore. </thinking>

Tool #1: websearch
<thinking> Based on the search results, Bedrock AgentCore appears to be a tool or service related to Amazon Bedrock, possibly for developing and managing AI agents or applications. The results mention using Bedrock AgentCore for various purposes, such as creating chat applications, developing promotional video generation agents, and implementing infrastructure as a prompt. I will summarize this information to provide an introduction to Bedrock AgentCore. </thinking>

Bedrock AgentCore is a component of Amazon Bedrock, likely used for developing and managing AI agents or applications. It has been mentioned in various contexts, such as creating chat applications, developing promotional video generation agents, and

## 在 Bedrock AgentCore Runtime 中部署带身份验证的 MCP 服务器

现在我们将配置 MCP 服务器并将其部署到 Bedrock AgentCore Runtime。此过程涉及创建依赖项、配置身份验证和部署服务。
![bedrock-agentcore-runtime-launch](images/runtime-launch.png)

### 步骤 1：设置 Amazon Cognito 身份验证

为已部署的 MCP 服务器创建 Cognito 用户池以实现安全访问。

创建的组件
- **用户池**：管理用户身份
- **应用客户端**：启用应用程序身份验证
- **测试用户**：用于测试身份验证流程

In [6]:
import boto3

region = boto3.session.Session().region_name

# Initialize Cognito client
cognito_client = boto3.client('cognito-idp', region_name=region)

# Create User Pool
user_pool_response = cognito_client.create_user_pool(
    PoolName='MCPServerPool',
    Policies={
        'PasswordPolicy': {
            'MinimumLength': 8
        }
    }
)
cognito_pool_id = user_pool_response['UserPool']['Id']

# Create App Client
app_client_response = cognito_client.create_user_pool_client(
    UserPoolId=cognito_pool_id,
    ClientName='MCPServerPoolClient',
    GenerateSecret=False,
    ExplicitAuthFlows=[
        'ALLOW_USER_PASSWORD_AUTH',
        'ALLOW_REFRESH_TOKEN_AUTH'
    ]
)
cognito_client_id = app_client_response['UserPoolClient']['ClientId']

# Create User
cognito_client.admin_create_user(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    TemporaryPassword='Temp123!',
    MessageAction='SUPPRESS'
)

# Set Permanent Password
cognito_client.admin_set_user_password(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    Password='MyPassword123!',
    Permanent=True
)


# Output the required values
print(f"Pool id: {cognito_pool_id}")
print(f"Discovery URL: https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration")
print(f"Client ID: {cognito_client_id}")

Pool id: us-east-1_7FN3X9nNU
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_7FN3X9nNU/.well-known/openid-configuration
Client ID: 2c29908qquvjcp9nmn701tehik


### 步骤 2：配置 Bedrock AgentCore Runtime

Set up the Bedrock AgentCore Runtime configuration with automatic resource creation.

**Generated Artifacts:**
This step creates essential deployment files:
- **Dockerfile**: Container configuration for the MCP Server
- **.dockerignore**: list the excluded files when docker build
- **.bedrock_agentcore.yaml**: Runtime deployment configuration

In [7]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3

region = boto3.session.Session().region_name
print(f"Using AWS region: {region}")

agentcore_runtime = Runtime()

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    protocol="MCP",
    agent_name="mcp_server_agentcore",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_client_id],
            "discoveryUrl": f"https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration",
        }
    }
)
print("Configuration completed ✓")

Entrypoint parsed: file=/workshop/sample-strands-in-5-minutes/bedrock-agentcore-integration/workshop/cn/03-bedrock-agentcore-runtime-mcp/mcp_server.py, bedrock_agentcore_name=mcp_server
Configuring BedrockAgentCore agent: mcp_server_agentcore


Using AWS region: us-east-1
Configuring AgentCore Runtime...


Generated .dockerignore
Generated Dockerfile: /workshop/sample-strands-in-5-minutes/bedrock-agentcore-integration/workshop/cn/03-bedrock-agentcore-runtime-mcp/Dockerfile
Generated .dockerignore: /workshop/sample-strands-in-5-minutes/bedrock-agentcore-integration/workshop/cn/03-bedrock-agentcore-runtime-mcp/.dockerignore
Setting 'mcp_server_agentcore' as default agent
Bedrock AgentCore configured: /workshop/sample-strands-in-5-minutes/bedrock-agentcore-integration/workshop/cn/03-bedrock-agentcore-runtime-mcp/.bedrock_agentcore.yaml


Configuration completed ✓


### 步骤 3：部署到 Bedrock AgentCore Runtime

使用 AWS CodeBuild 启动部署过程以进行容器化和部署。

**部署过程：**
- 构建 MCP 服务器的容器化版本
- 创建所需的 AWS 资源（ECR 存储库、IAM 角色）
- 将容器镜像推送到 Amazon ECR
- 作为托管的自动扩展服务部署到 AgentCore Runtime

In [8]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_agentcore' to account 710299592439 (us-east-1)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_agentcore' to account 710299592439 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: mcp_server_agentcore
✅ ECR repository available: 710299592439.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-mcp_server_agentcore
Getting or creating execution role for agent: mcp_server_agentcore
Using AWS region: us-east-1, account ID: 710299592439
Role name: AmazonBedrockAgentCo

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-mcp_server_agentcore


Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
Starting execution role creation process for agent: mcp_server_agentcore
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
✓ Role created: arn:aws:iam::710299592439:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-mcp_server_agentcore
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::710299592439:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: mcp_server_agentcore
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-7f3ae149b4
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-7f3ae149b4
Creating IAM

### 验证部署状态

检查部署状态并等待运行时准备就绪：

In [9]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

mcp_runtime_id = launch_result.agent_id
mcp_runtime_arn = launch_result.agent_arn
mcp_ecr_repo_name = launch_result.ecr_uri.split('/')[1]
mcp_codebuild_name = launch_result.codebuild_id.split(':')[0]
print(f"MCP AgentCore Runtime ID: {mcp_runtime_id}")
print(f"MCP AgentCore Runtime ARN: {mcp_runtime_arn}")
print(f"ECR Repo for MCP AgentCore Runtime: {mcp_ecr_repo_name}")
print(f"CodeBuild Project for MCP AgentCore Runtime: {mcp_codebuild_name}")

Checking AgentCore Runtime status...


Retrieved Bedrock AgentCore status for: mcp_server_agentcore


Initial status: READY
✓ AgentCore Runtime is READY!
Final status: READY
MCP AgentCore Runtime ID: mcp_server_agentcore-qsyNr2G414
MCP AgentCore Runtime ARN: arn:aws:bedrock-agentcore:us-east-1:710299592439:runtime/mcp_server_agentcore-qsyNr2G414
ECR Repo for MCP AgentCore Runtime: bedrock-agentcore-mcp_server_agentcore
CodeBuild Project for MCP AgentCore Runtime: bedrock-agentcore-mcp_server_agentcore-builder


### 使用 Strands Agent 测试已部署的 MCP 服务器作为工具

现在让我们通过 Bedrock AgentCore Runtime 端点连接到已部署的 MCP 服务器并进行适当的身份验证来测试它。

In [10]:
import boto3

region = boto3.session.Session().region_name

# Get bearer token (access token) from Cognito Auth 
cognito_client = boto3.client('cognito-idp', region_name=boto3.session.Session().region_name)
auth_response = cognito_client.initiate_auth(
    ClientId=cognito_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': 'MyPassword123!'
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']
print(bearer_token)

eyJraWQiOiJmd2xuN0pFSGp1REdISkJTdWJXaEFzYnpOd3Zoa1E1bGRjZlErdVVcL2Zndz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJhNGU4OTQ4OC1jMDMxLTcwMmUtMmJkMy1kZWNlNWQwNTY5ZDEiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV83Rk4zWDluTlUiLCJjbGllbnRfaWQiOiIyYzI5OTA4cXF1dmpjcDlubW43MDF0ZWhpayIsIm9yaWdpbl9qdGkiOiI2NGYyMjViZC1jYmQwLTQ3YjUtYmYzOC1hNTg1MzVlMGQxMTMiLCJldmVudF9pZCI6ImY5ZmFkN2JjLWRkMWYtNDFhYi04ZGEzLTJlMGViMGE0MjljNiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NTYzNTAzNjMsImV4cCI6MTc1NjM1Mzk2MywiaWF0IjoxNzU2MzUwMzYzLCJqdGkiOiI4NmRlNTNmNS1jOTAxLTQ1NzctOTdlYy1mZjg5NjZkZjNhNWYiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.Hx3f_USrL-sCun23lYuaIYihq5OInYOxWipwrlLt5CZ4XIMbx_OoSN6My40y7hL5jUV_ee0BfGUUz-WjwPjiaLjkLJjoWwEyKYaN9bl7qhzbrVKamACIirszAIdPPWQzEyQMgqpjGABjx6IeEqYIa7TRbZ6hG6FE2SGKFBhROml5_Z6XDlQt0WTdPQm7AfOADhWBfreCildhwO8E2LFGnVkoAI2mZhpzBaUTk4SiUsTDavhDnuF4cv1IQjSQx8-FPjJk7v3IJbv27cOKf26GEE2K7yheSoh6csWQ9Y3l3dR-5boiWCg

In [12]:
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
#import region

region = boto3.session.Session().region_name

mcp_runtime_arn = launch_result.agent_arn
encoded_arn = mcp_runtime_arn.replace(':', '%3A').replace('/', '%2F')

# Connect to the Web Search MCP server
print("\nConnecting to MCP Server...")
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    #"Content-Type": "application/json"
}
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url, headers))

with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"Available tools: {[tool.tool_name for tool in mcp_tools]}")

    # Create agent with custom MCP tools
    agent = Agent(
        model="us.amazon.nova-pro-v1:0",  # Optional: Specify the model ID
        system_prompt="You are a helpful assistant that provides concise responses.",
        tools=mcp_tools,
    )

    response = agent("介绍 Bedrock AgentCore 是甚么")
    print(response) 


Connecting to MCP Server...


Available tools: ['websearch']
<thinking> To introduce Bedrock AgentCore, I need to search for relevant information about it. I will use the websearch tool to find information about Bedrock AgentCore. </thinking>

Tool #1: websearch
Bedrock AgentCore 是由亚马逊提供的一项服务，旨在帮助开发者安全、大规模地部署和运行高效的人工智能代理。它提供了专为动态代理工作负载设计的基础设施、增强代理功能的强大工具，以及实际部署所需的基本控制。Bedrock AgentCore 旨在加速人工智能代理

Session termination failed: 404


进入生产环境，提供所需的规模、可靠性和安全性。Bedrock AgentCore 是由亚马逊提供的一项服务，旨在帮助开发者安全、大规模地部署和运行高效的人工智能代理。它提供了专为动态代理工作负载设计的基础设施、增强代理功能的强大工具，以及实际部署所需的基本控制。Bedrock AgentCore 旨在加速人工智能代理进入生产环境，提供所需的规模、可靠性和安全性。



Let's examine the detailed execution flow of the agent loop to understand how the agent processes requests and generates responses:

In [13]:
print("Agent Loop Detail")
print("-----------------")

print(f"Agent Loop Length: {len(agent.messages)})")
print("\nUser-Assistant Conversation:")
for message in agent.messages:
    print(message)

Agent Loop Detail
-----------------
Agent Loop Length: 4)

User-Assistant Conversation:
{'role': 'user', 'content': [{'text': '介绍 Bedrock AgentCore 是甚么'}]}
{'role': 'assistant', 'content': [{'text': '<thinking> To introduce Bedrock AgentCore, I need to search for relevant information about it. I will use the websearch tool to find information about Bedrock AgentCore. </thinking>\n'}, {'toolUse': {'toolUseId': 'tooluse_RPt_Mw76QO2S3tGfXTPAsw', 'name': 'websearch', 'input': {'keywords': 'Bedrock AgentCore', 'max_results': '5', 'region': 'us-en'}}}]}
{'role': 'user', 'content': [{'toolResult': {'status': 'success', 'toolUseId': 'tooluse_RPt_Mw76QO2S3tGfXTPAsw', 'content': [{'text': '{\n  "title": "Amazon Web Services Amazon Bedrock AgentCore (Preview) - AWS",\n  "href": "https://aws.amazon.com/bedrock/agentcore/",\n  "body": "1 week ago - Amazon Bedrock AgentCore enables you to deploy and operate highly capable AI agents securely, at scale . It offers infrastructure purpose-built for dyna

## 资源清理（可选）

清理已部署的资源：

In [14]:
import boto3
import os

region = boto3.session.Session().region_name

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr',region_name=region)
codebuild_client = boto3.client('codebuild',region_name=region)
cognito_client = boto3.client('cognito-idp', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    agentcore_control_client.delete_agent_runtime(agentRuntimeId=mcp_runtime_id)
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_client.delete_repository(repositoryName=mcp_ecr_repo_name, force=True)
    print("✓ ECR repository deleted")

    print("Deleting CodeBuild Project...")
    codebuild_client.delete_project(name=mcp_codebuild_name)
    print("✓ CodeBuild Project deleted")

    print("Deleting Cognito User Pool...")
    cognito_client.delete_user_pool(UserPoolId=cognito_pool_id)
    print("✓ Cognito User Pool deleted")

    print("Deleting Bedrock AgentCore configuration file...")
    os.remove(".bedrock_agentcore.yaml") 
    print("✓ .bedrock_agentcore.yaml deleted")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

Deleting AgentCore Runtime...
✓ AgentCore Runtime deletion initiated
Deleting ECR repository...
✓ ECR repository deleted
Deleting CodeBuild Project...
✓ CodeBuild Project deleted
Deleting Cognito User Pool...
✓ Cognito User Pool deleted
Deleting Bedrock AgentCore configuration file...
✓ .bedrock_agentcore.yaml deleted
